In [1]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
import time
import pandas as pd

In [2]:
def get_jobs(keyword, num_jobs, verbose):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path="C:/Users/khairie/Anaconda3/chromedriver", options=options)
    driver.set_window_size(1120, 1000)

    url ='https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=Data+Scientist&sc.keyword=Data+Scientist&locT=&locId=&jobType='
    #url = 'https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword=&locT=C&locId=2986682&jobType=&context=Jobs&sc.keyword=Data+Scientist&dropdown=0'
    #url = 'https://www.glassdoor.com/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword="+keyword+"&locId=&jobType=&context=Jobs&sc.keyword=Data+Science&dropdown=0' #malaysia
    driver.get(url)
    jobs = []

    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(4)

        #Test for the "Sign Up" prompt and get rid of it.
        try:
            driver.find_element_by_class_name("selected").click()
        except ElementClickInterceptedException:
            pass

        time.sleep(.1)

        try:
            driver.find_element_by_css_selector('[alt="Close"]').click()  #clicking to the X.
        except NoSuchElementException:
            pass

        
        #Going through each job in this page
        job_buttons = driver.find_elements_by_class_name("jl")  #jl for Job Listing. These are the buttons we're going to click.
        for job_button in job_buttons:  

            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break

            driver.execute_script("arguments[0].click();", job_button)  #You might 
            time.sleep(1)
            collected_successfully = False
            
            while not collected_successfully:
                try:
                    company_name = driver.find_element_by_xpath('.//div[@class="employerName"]').text
                    location = driver.find_element_by_xpath('.//div[@class="location"]').text
                    job_title = driver.find_element_by_xpath('.//div[contains(@class, "title")]').text
                    job_description = driver.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text
                    collected_successfully = True
                except:
                    time.sleep(5)

            try:
                salary_estimate = driver.find_element_by_xpath('.//span[@class="gray salary"]').text
            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."
            
            try:
                rating = driver.find_element_by_xpath('.//span[@class="rating"]').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            try:
                driver.find_element_by_xpath('.//div[@class="tab" and @data-tab-type="overview"]').click()

                try:
                    #<div class="infoEntity">
                    #    <label>Headquarters</label>
                    #    <span class="value">San Francisco, CA</span>
                    #</div>
                    headquarters = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*').text
                except NoSuchElementException:
                    headquarters = -1

                try:
                    size = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Size"]//following-sibling::*').text
                except NoSuchElementException:
                    size = -1

                try:
                    founded = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Founded"]//following-sibling::*').text
                except NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Type"]//following-sibling::*').text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Industry"]//following-sibling::*').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Sector"]//following-sibling::*').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Revenue"]//following-sibling::*').text
                except NoSuchElementException:
                    revenue = -1

                try:
                    competitors = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Competitors"]//following-sibling::*').text
                except NoSuchElementException:
                    competitors = -1

            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                headquarters = -1
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1
                competitors = -1

                
            if verbose:
                print("Headquarters: {}".format(headquarters))
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("Competitors: {}".format(competitors))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Headquarters" : headquarters,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue,
            "Competitors" : competitors})
            #add job to jobs

        #Clicking on the "next page" button
        try:
            driver.find_element_by_xpath('.//li[@class="next"]//a').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.

In [3]:
df = get_jobs("data scientist", 10, False)
df

Progress: 0/10
Progress: 1/10
Progress: 2/10
Progress: 3/10
Progress: 4/10
Progress: 5/10
Progress: 6/10
Progress: 7/10
Progress: 8/10
Progress: 9/10
Progress: 10/10


,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Data Scientist,$78K-$133K (Glassdoor est.),"Publishers Clearing House (PCH), located in Je...",3.1,Publishers Clearing House\n3.1,"Jericho, NY",-1,501 to 1000 Employees,1953,Company - Private,Advertising & Marketing,Business Services,$500 million to $1 billion (USD),-1
1,Data Scientist,$78K-$133K (Glassdoor est.),Description\n\nPosition at Homes.com\n\nHomes....,4.2,Homes.com\n4.2,"Norfolk, VA",-1,201 to 500 Employees,1998,Company - Private,Real Estate,Real Estate,$50 to $100 million (USD),-1
2,Data Scientist Virtual Hiring Event,$78K-$133K (Glassdoor est.),LMI Hiring Event!\n\nEvent Details\nDate: Wedn...,4.2,LMI\n4.2,"McLean, VA",-1,1001 to 5000 Employees,1961,Nonprofit Organization,Consulting,Business Services,$100 to $500 million (USD),-1
3,Data Analyst,$78K-$133K (Glassdoor est.),"Rational is growing, and we need amazing talen...",2.7,Rational\n2.7,"Redmond, WA",-1,201 to 500 Employees,2009,Company - Public,Consulting,Business Services,Unknown / Non-Applicable,-1
4,Data Scientist,$78K-$133K (Glassdoor est.),Description\n\nJob Description:\n\nLeidos has ...,3.6,Leidos\n3.6,"McLean, VA",-1,10000+ Employees,1969,Company - Public,Aerospace & Defense,Aerospace & Defense,$10+ billion (USD),-1
5,Data Engineer Virtual Hiring Event,$78K-$133K (Glassdoor est.),LMI Hiring Event!\n\nEvent Details\nDate: Wedn...,4.2,LMI\n4.2,"McLean, VA",-1,1001 to 5000 Employees,1961,Nonprofit Organization,Consulting,Business Services,$100 to $500 million (USD),-1
6,Senior Data Scientist Virtual Hiring Event,$78K-$133K (Glassdoor est.),The Zebra Hiring Event // Senior Data Scientis...,4.0,The Zebra\n4.0,"Austin, TX",-1,51 to 200 Employees,2012,Company - Private,Insurance Agencies & Brokerages,Insurance,Unknown / Non-Applicable,-1
7,Data Engineer,$78K-$133K (Glassdoor est.),About Conduent:\n\nConduent delivers mission-c...,2.8,Conduent\n2.8,"Morrisville, NC",-1,10000+ Employees,2017,Company - Public,IT Services,Information Technology,Unknown / Non-Applicable,-1
8,Data Scientist 1,$78K-$133K (Glassdoor est.),"At NGL, we strive to make data-driven decision...",3.2,National Guardian Life Insurance Company\n3.2,"Madison, WI",-1,201 to 500 Employees,1909,Company - Private,Insurance Carriers,Insurance,$500 million to $1 billion (USD),-1
9,Data Analyst,$78K-$133K (Glassdoor est.),Data Analyst\n\nJob Details\nJob Location\nYak...,3.3,Comprehensive Healthcare\n3.3,"Yakima, WA",-1,501 to 1000 Employees,1971,Nonprofit Organization,Health Care Services & Hospitals,Health Care,Unknown / Non-Applicable,-1
